In [2]:
import os, subprocess
import json
import uproot
import awkward as ak
import numpy as np
from coffea import processor, util, hist
import pickle
import pandas as pd

#Plot settings
%matplotlib inline
import matplotlib.pyplot as plt
import mplhep as hep
plt.style.use(hep.style.CMS)

import matplotlib.pylab as pylab
params = {'legend.fontsize': 'medium',
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'medium',
         'ytick.labelsize':'medium'}
pylab.rcParams.update(params)

#line thickness
import matplotlib as mpl
#mpl.rcParams['lines.linewidth'] = 5

#Set color scheme
global color_dict 
color_dict = {}
color_dict['QCD'] = 'black'
color_dict['Wjets'] = 'gray'
color_dict['Zjets'] = 'deepskyblue'
color_dict['Zjetsbb'] = 'blue'
color_dict['ttbar'] = 'purple'
color_dict['singlet'] = 'hotpink'
color_dict['VV'] = 'darkorange'

color_dict['ttH'] = 'purple'
color_dict['WH'] = 'blue'
color_dict['ZH'] = 'gold'
color_dict['VBF'] = 'green'
color_dict['ggF'] = 'red'

#Dataset parameters
lumis = {}
lumis['2016'] = 35.9
lumis['2017'] = 41.5
lumis['2018'] = 59.9


systematics = ['nominal',
               'jet_triggerUp','jet_triggerDown',
               'btagWeightUp','btagWeightDown','btagEffStatUp','btagEffStatDown',
               'UESUp','UESDown','JESUp','JESDown','JERUp','JERDown',
              ]

mc = ['QCD','Wjets','Zjets','ttbar','singlet','VV','ggF','VBF','ZH','WH']


#To calculate significance
def significance(s,b):
    if b==0:
        return 0
    z_squared = 2.0*(s+b)*np.log(1.0+1.0*s/b) - 2.0*s
    return np.sqrt(z_squared)

In [ ]:
def get_data(file_path):
    
    process_list = ['QCD',
                    'Wjets',
                    'Wjetsbb',
                    'Zjets',
                    'Zjetsbb',
                    'ttbar',
                    'singlet',
                    'VV',
                    'ggF',
                    'VBF',
                    'ZH',
                    'WH',
                    'ttH']
    
    

In [3]:
!ls ../2017

1mv-signalregion.root	1mvl-signalregion.root
1mvc-signalregion.root	templates.pkl


In [6]:
a = uproot.open("../2017/1mv-signalregion.root")

In [19]:
a['pass_mv1_ZH_nominal']

<TH1D (version 3) at 0x7f9e028f6f10>

In [8]:
a['pass_mv1_data_nominal']

<b'TH1D' b'pass_mv1_data_nominal' 0x7fee387ac860>

In [ ]:
def plot_mconly_inc(h, name, title, log=True):
    fig = plt.figure()
    plt.suptitle(title)

    ax = fig.add_subplot(111)

    # https://matplotlib.org/stable/gallery/color/named_colors.html                                                                     
    labels = ['VV','single t','tt','W+jets','Z+jets','QCD']
    mc = ['QCD','Wjets','Zjets','ttbar','singlet','VV']
    colors=['gray','deepskyblue','blue','purple','hotpink','darkorange']

    if log:
        mc = [x for x in reversed(mc)]
        colors = [x for x in reversed(colors)]
        labels = [x for x in reversed(labels)]

    # Plot stacked hist                                                                                                                
    hist.plot1d(h,order=mc,stack=True,fill_opts={'color':colors,'edgecolor':'black'})

    sig = h.integrate('process',['ggF','VBF','WH','ZH','ttH'])
    hist.plot1d(sig,stack=False,line_opts={'color':'green'})
    labels = labels + ['Higgs']
    plt.legend(labels,bbox_to_anchor=(1.05, 1), loc='upper left')
    ax.set_ylim(0,1000000)

    if log:
        ax.set_yscale('log')
        ax.set_ylim(0.1,10000000)

    # save as name                                                                                                                  
    #png_name = name+'.png'
    #plt.savefig(png_name,bbox_inches='tight')

    #pdf_name = name+'.pdf'
    #plt.savefig(pdf_name,bbox_inches='tight')